In [1]:
import copy

import numpy as np
import matplotlib.pyplot as plt
import os as os

path = "./logs/"

The load_curves function loads the results of multiple algorithms across different environments from .npy files, stores them in a dictionary structure, and processes them to ensure the curves all have the same length.

In [2]:
def load_curves(algorithms, envs):
    curves = {alg: {e: [] for e in envs} for alg in algorithms} # create a dictionary 
    for alg in algorithms:
        for e in envs:
            for i in range(0, 100):
                name = f"{path}{e}/{alg}/{alg}_{i}.npy"
                if os.path.isfile(name):
                    # print(f"{name} loaded", np.load(name).shape)
                    curves[alg][e].append(list(np.load(name)))
            max_size = -1        
            for i in range(len(curves[alg][e])):
                if len(curves[alg][e][i]) > max_size:
                    max_size = len(curves[alg][e][i])
            for i in range(len(curves[alg][e])):
                while len(curves[alg][e][i]) <= max_size:
                    curves[alg][e][i].append(curves[alg][e][i][-1])

            curves[alg][e] = np.array(curves[alg][e]) if len(curves[alg][e]) else np.array([[]])


    return curves

The smooth function applies a smoothing technique to the loaded data by averaging over a window of neighboring points, creating a more visually appealing curve by reducing fluctuations.

In [3]:
def smooth(data, length=10):
    data = data.copy()
    pad = copy.deepcopy(data[:, -1, None].repeat(length-1, axis=-1)) # padding the data
    pad_data = np.concatenate((data, pad), axis=-1)
    pad = copy.deepcopy(data[:, 0, None].repeat(length-1, axis=-1))  # padding the data
    pad_data = np.concatenate((pad, pad_data), axis=-1)
    
    for i in range(length, len(data[0])+length):
        assert i-length >= 0
        data[:, i-length] = np.mean(pad_data[:, i-length:i+length], axis=-1) # smooth the data with its average
    return data

Plots the performance curves for each algorithm on each environment.

In [ ]:
# Define environments and algorithms
ENVS = ['MobileRobotEnv']
ALGORITHMS = ['DDPG']

# Load the curves
curves = load_curves(ALGORITHMS, ENVS)


# Plotting Results
for k, e in enumerate(ENVS):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.set_title(e)  # Set environment name as the title
    for i, alg in enumerate(ALGORITHMS):
        c = curves[alg][e]  # Get data for the algorithm on this environment
        if c.size == 0:
            print(f"No data for {alg} on {e}. Skipping.")
            continue  # Skip if there's no data
        mu = smooth(c.mean(axis=0, keepdims=True), length=2).squeeze()  # Smooth mean
        sigma = smooth(c.std(axis=0, keepdims=True), length=2).squeeze()  # Smooth std deviation
        x = np.arange(mu.shape[0]) # / 2 modifed for represent the true number of episodes  # X-axis (time or episode count)
        
        ax.plot(x, mu, label=alg, linewidth=4, linestyle='--', color='C' + str(i))  # Plot mean curve
        ax.fill_between(x, mu + sigma, mu - sigma, alpha=0.2, color='C' + str(i))  # Fill between std deviation
        ax.grid(True)  # Add grid
    
    ax.legend()  # Add legend for algorithms
    plt.show()  # Show the plot

# ax.set_ylim([0, 2000])
